# 0.0 Modules, etc.

In [2]:
import numpy as np
import pandas as pd
import multiprocessing

import scipy
from scipy import spatial
from scipy.spatial import cKDTree

import sklearn as sk
from sklearn import svm
from sklearn import preprocessing
from sklearn import neighbors
from sklearn.preprocessing import StandardScaler
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.feature_selection import RFECV
from sklearn import clone
from sklearn.externals.six.moves import xrange

import matplotlib.pyplot as plt
import bokeh
import bokeh.io
from bokeh.plotting import figure
from bokeh.io import output_notebook, show

# init_notebook_mode()

import seaborn as sns

import re
import math
import copy

from collections import defaultdict
import csv
import itertools
import datetime 
from datetime import datetime
import time
import dateutil.parser
import pickle
import random

import gc
import zipfile
import sys, getopt
import os

from IPython.core.interactiveshell import InteractiveShell
from io import StringIO

import dask.dataframe as dd
#from chest import Chest

InteractiveShell.ast_node_interactivity = "all"
#InteractiveShell.ast_node_interactivity = "last"

# Magic function to make matplotlib inline
%matplotlib inline

%config InlineBackend.figure_formats = {'png', 'retina'}

# Set up Bokeh for inline viewing
bokeh.io.output_notebook()

import dask.dataframe as ddf
import dask.array as da

Loading BokehJS ...

In [3]:
pd.set_option('max_columns', 500)
pd.set_option('max_rows', 500)

In [ ]:
import plotly
import plotly.plotly as py
import plotly.graph_objs as go
# plotly.tools.set_credentials_file(username='duplinskiy', api_key='RsZHhxIiAGGu7FN9P4bu')
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import cufflinks as cf

cf.go_offline()

### Useful example article -- C77124

In [ ]:
dat[dat['article_number'] == 'C77124']


    - Small: < 100 units
    - Large: > 30000 units

### MAPE: FW17, SS18, FW18

In [ ]:
preds = (pd.DataFrame(
    pd.read_csv('Buyers predictions.csv', low_memory=False, index_col = 0))
         [['season', 'ecom_marketing_forecast']].
         reset_index().
        dropna())

In [ ]:
# dat = pd.read_csv('dat.csv', low_memory=False, index_col = 0) # Wall time: 1min 47s
# dat['consumer_order_date'] = pd.to_datetime(dat['consumer_order_date'])
# dat = dat[dat['gross_demand_quantity'] != 0] 
# dat.drop(['sales_organization', 'country', 'brand', 'sold_qty', 'std_margin', 
#           'return_qty', 'article_promotion_main_category_group', 'fw_or_ss'], 
#          inplace=True, axis = 1)
# dat = dat.groupby(by = ['article_number', 'season']).agg('sum')[['net_qty']] # aggregate by year
# dat.reset_index(inplace=True)

In [ ]:
# dat.to_csv('dat_raw_MAPE.csv')
dat = pd.read_csv('dat_raw_MAPE.csv', low_memory=False, index_col = 0,) # to calculate raw MAPE


In [ ]:
# Merge predictions
dat1 = pd.merge(dat, preds, left_on=['article_number', 'season'], right_on=['article', 'season'])
dat1 = dat1[dat1['season'] != 'SS19'].drop('article', axis = 1)
dat1 = dat1[(dat1['net_qty'] != 0) & (dat1['ecom_marketing_forecast'] != 0)]
del dat

In [ ]:
%whos DataFrame

# dat.head() # 47756 
# preds.head()

dat1.shape
dat1.head()

In [ ]:
# pct error = [forecast - actual]/actual = (net_qty + stock - net_qty)/net_qty
dat1['APE'] = abs(dat1['ecom_marketing_forecast'] - dat1['net_qty'])/dat1['net_qty']*100

In [ ]:
bins = [0, 100, 10000, 30000]
dat1['bin'] = pd.cut(np.array(dat1['net_qty']), bins)

d = dat1.groupby(['season', 'bin'])['APE'].describe()[['count', 'mean', '50%']]

order = {'SS17': 0, 'FW17': 1, 'SS18': 2, 'FW18': 3, 'SS19': 4}
d['order_id'] = [order[i] for i in d.reset_index()['season']]

d.sort_values(by = ['order_id', 'bin'], inplace=True)
d.drop('order_id', axis = 1, inplace=True)

d.round().astype('int')

# Corrected-MAPE

### Understock EDA

In [ ]:
carryover = pd.read_csv('Stock left and of season.csv', low_memory=False, index_col = 0, usecols=['season', 'article_number', 'ecom_available_stock', 'buy_availability']) 
carryover = (pd.DataFrame(carryover.groupby(['article_number', 'season'])['ecom_available_stock'].min()).reset_index())

dat0 = pd.read_csv('dat_raw_MAPE.csv', low_memory=False, index_col = 0) 


In [ ]:
dat = dat0.copy()

In [ ]:
dat = pd.merge(dat, carryover, left_on=['article_number', 'season'], right_on=['article_number', 'season']) # Add end of season stock information
dat_understock = dat[dat['ecom_available_stock'] == 0] # leftover was 0, so understocked 
dat_understock = dat_understock[dat_understock['season'] != 'SS19'] # remove current season
dat_understock = dat_understock[dat_understock['net_qty'] > 100] # remove small potatoes items


In [ ]:
# --- Sort & order by season ---
bins = [0, 100, 1000, 30000]
dat_understock['bin'] = pd.cut(np.array(dat_understock['net_qty']), bins)

d = pd.DataFrame(dat_understock.groupby(['season', 'bin'])['article_number'].describe()['count'])

order = {'SS17': 0, 'FW17': 1, 'SS18': 2, 'FW18': 3, 'SS19': 4}
d['order_id'] = [order[i] for i in d.reset_index()['season']]

d.sort_values(by = ['order_id', 'bin'], inplace=True)
d.drop('order_id', axis = 1, inplace=True)
d

### Understock Correction

In [ ]:
# dat = pd.read_csv('dat.csv', low_memory=False, index_col = 0) 
# dat['consumer_order_date'] = pd.to_datetime(dat['consumer_order_date'])
# dat = dat[dat['net_qty'] != 0] 
# dat = dat[dat['season'] != 'SS19']
# dat.drop(['sales_organization', 'country', 'brand', 'sold_qty', 'gross_demand_quantity', 'net_sales', 'std_margin', 'return_qty', 'article_promotion_main_category_group', 'fw_or_ss'], inplace=True, axis = 1)

# dat.set_index('consumer_order_date', inplace = True)
# dat = dat.groupby(['article_number', 'season']).resample('W').sum() # 'aggregate' to weekly sums by article
# dat.reset_index(inplace=True)
# dat['consumer_order_date'] = pd.to_datetime(dat['consumer_order_date'])
# dat['week'] = [t.week for t in dat['consumer_order_date']]
# dat['year'] = [t.year for t in dat['consumer_order_date']]

In [ ]:
%%time

# dat.to_csv('weekly_dat.csv')
dat = pd.read_csv('weekly_dat.csv', low_memory=False, index_col = 0)
dat['consumer_order_date'] = pd.to_datetime(dat['consumer_order_date'])
dat = dat[dat['season'] != 'SS19']

In [ ]:
# ******* THIS IS AN ISSUE ******* MISSING ~20K ARTICLES *********
buy_avail = pd.read_csv('stock data per week and per article.csv', low_memory=False) # 

# buy_avail['min_date_of_week'] = pd.to_datetime(buy_avail['min_date_of_week'])
# buy_avail = buy_avail[['article_number', 'min_date_of_week', 'avg(buy_availability)']]
# buy_avail['week'] = [t.week for t in buy_avail['min_date_of_week']]
# buy_avail['year'] = [t.year for t in buy_avail['min_date_of_week']]


In [ ]:
len(buy_avail['article_number'].unique()) # ******* THIS IS AN ISSUE *******

In [ ]:
# buy_avail.to_csv('buy_availability.csv')
buy_avail = pd.read_csv('buy_availability.csv', low_memory=False, index_col = 0)
buy_avail = buy_avail[(buy_avail['year'] != 2019) & (buy_avail['year'] != 2016)]

In [ ]:
datA = pd.merge(dat, buy_avail, # THIS IS AN INNER JOIN. PROBLEM?
                left_on = ['article_number', 'year', 'week'], 
                right_on = ['article_number', 'year', 'week'],
               )

# datB = pd.merge(dat, buy_avail, 
#                 left_on = ['article_number', 'year', 'week'], 
#                 right_on = ['article_number', 'year', 'week'],
#                 how = 'outer')

datA.shape # 719 653
datB.shape # 2 290 052

In [ ]:
datA[datA['article_number'] == 'CF1324']
datB[datB['article_number'] == 'CF1324']

In [ ]:
# Add indicator for buy availability > 0.35
dat['stockout_indicator'] = dat['avg(buy_availability)'] < 0.35
# dat['stockout_indicator'].mean() # 0.1706

In [ ]:
season_dat = pd.DataFrame(
    dat.
    groupby(['article_number', 'season'])['net_qty'].
    sum()
)

# Season averages for articles when fully stocked
stocked = pd.DataFrame(
    dat[dat['stockout_indicator'] == False].
    groupby(['article_number', 'season'])['net_qty'].
    mean()
)

stocked['season_projected'] = 26*stocked['net_qty']

season_dat = (
    pd.merge(season_dat, stocked, left_index=True, right_index=True, how = 'outer').
    rename(columns = {'net_qty_x':'season_actual', 'net_qty_y':'daily_available_avg'})
)

del stocked

In [ ]:
preds = (pd.DataFrame(
    pd.read_csv('Buyers predictions.csv', low_memory=False, index_col = 0))
         [['season', 'ecom_marketing_forecast']].
         reset_index().
        dropna())

season_dat = pd.merge(
    season_dat, 
    preds, 
    left_index=True, 
    right_on=['article', 'season'], 
    how = 'outer').round().set_index(['article', 'season'])

season_dat['season*'] = season_dat[['season_projected', 'season_actual']].max(axis = 1)
season_dat['APE'] = abs(season_dat['season*'] - season_dat['ecom_marketing_forecast'])/season_dat['season*']*100

season_dat.reset_index(inplace = True)
season_dat.dropna(inplace = True)

del preds

In [ ]:
bins = [0, 100, 1000, 30000]
season_dat['bin'] = pd.cut(np.array(season_dat['season_actual']), bins) # net_qty

d = season_dat.groupby(['season', 'bin'])['APE'].describe()[['count', 'mean', '50%']]

# --- Sort by season ---

order = {'SS17': 0, 'FW17': 1, 'SS18': 2, 'FW18': 3, 'SS19': 4}
d['order_id'] = [order[i] for i in d.reset_index()['season']]

d.sort_values(by = ['order_id', 'bin'], inplace=True)
d.drop('order_id', axis = 1, inplace=True)

d.round()

In [ ]:
%whos DataFrame 

### C-MAPE: Overstock EDA

In [ ]:
# clearance = pd.read_csv('Sale Data/Sale data FW17.csv', low_memory=False, index_col = 0) 

# clearance = clearance.reset_index() # groupby('article_number').resample('W')
# clearance['consumer_order_date'] = pd.to_datetime(clearance['consumer_order_date'])
# clearance.set_index('consumer_order_date', inplace=True)

In [ ]:
dat0 = pd.read_csv('dat.csv', low_memory=False, index_col = 0) # Wall time: 1min 47s

In [ ]:
# Johannes said 'SALE' is Clearance, and outlet won't contribute much
SALE = dat0[dat0['article_promotion_main_category_group'] == 'SALE']

SALE['consumer_order_date'] = pd.to_datetime(SALE['consumer_order_date'])
SALE.set_index('consumer_order_date', inplace = True)

SALE2 = SALE[['article_number', 'season', 'net_qty']].groupby(['article_number', 'season']).resample('W').sum() # 'aggregate' to weekly sums by article
SALE2.reset_index(inplace=True)

In [ ]:
# HISTOGRAM
plt.rcParams["figure.figsize"] = [18,4]
SALE2['consumer_order_date'].hist(bins=50)

In [ ]:
SALE2.groupby(['consumer_order_date'])['net_qty'].sum().plot()

In [ ]:
# --- Outlet ---
outlet = dat0[dat0['article_promotion_main_category_group'] == 'Outlet']

outlet['consumer_order_date'] = pd.to_datetime(outlet['consumer_order_date'])
outlet.set_index('consumer_order_date', inplace = True)

outlet2 = outlet[['article_number', 'season', 'net_qty']].groupby(['article_number', 'season']).resample('W').sum()
outlet2.reset_index(inplace = True)

In [ ]:
plt.rcParams["figure.figsize"] = [18,4]
outlet2['consumer_order_date'].hist(bins = 50)

### C-MAPE over/stock plot

- Spring Summer: (SS) 1 December -- 31 May
- Fall Winter: (FW) 1 June -- 30 November

In [ ]:
# dat0 = pd.read_csv('dat.csv', low_memory=False, index_col = 0) # Wall time: 1min 47s

# dat = dat0.copy()
# dat = dat[['consumer_order_date', 'article_number', 
#             'net_qty', 'article_promotion_main_category_group',
#            'season']]
# dat['consumer_order_date'] = pd.to_datetime(dat['consumer_order_date'])

# dat['clearance'] = (dat['article_promotion_main_category_group'] == 'SALE')
# dat.drop('article_promotion_main_category_group', inplace = True, axis = 1)

In [ ]:
# dat.to_csv('dat_clearance.csv')
dat = pd.read_csv('dat_clearance.csv', low_memory=False, index_col = 0) 
dat['consumer_order_date'] = pd.to_datetime(dat['consumer_order_date'])

In [ ]:
dat_wo_clearance = dat.copy()
dat_wo_clearance = dat_wo_clearance[dat_wo_clearance['clearance'] != True] # **************
dat_wo_clearance = pd.DataFrame(dat_wo_clearance.groupby(['article_number', 'consumer_order_date'])['net_qty'].sum())
dat_wo_clearance = dat_wo_clearance.reset_index('article_number').groupby('article_number').resample('W').sum()
dat_wo_clearance = pd.DataFrame(dat_wo_clearance.reset_index().groupby('consumer_order_date')['net_qty'].mean())

In [ ]:
dat.head()
dat_wo_clearance_clearance.head()

In [ ]:
# ------- Clearance and non-clearance -------
dat_both = (pd.merge(dat, dat_wo_clearance, left_on='consumer_order_date', right_index=True).
            rename(columns = {'net_qty_y':'Without Clearance', 'net_qty_x':'With Clearance'})
           )

# With/without Clearance weekly average net_qty 
plt.rcParams["figure.figsize"] = [18,4]
dat_both.plot()

## w/o Clearance Correction

In [ ]:
%%time

# dat.to_csv('dat_clearance.csv')
dat = pd.read_csv('dat_clearance.csv', low_memory=False, index_col = 0) 
dat['consumer_order_date'] = pd.to_datetime(dat['consumer_order_date'])

In [ ]:
dat_wo_clearance = dat.copy()
dat_wo_clearance = dat_wo_clearance[dat_wo_clearance['clearance'] != True]
dat_wo_clearance = pd.DataFrame(dat_wo_clearance.groupby(['article_number', 'season'])['net_qty'].sum())

In [ ]:
preds = (pd.DataFrame(
    pd.read_csv('Buyers predictions.csv', low_memory=False, index_col = 0))
         [['season', 'ecom_marketing_forecast']].
         reset_index().
        dropna())

In [ ]:
season_dat = (pd.merge(
    dat_wo_clearance, preds, 
    left_index=True, right_on=['article', 'season'], 
    how = 'outer').
              round().
              # set_index(['article', 'season']).
              dropna()
             )
season_dat = season_dat[season_dat['season'] != 'SS19']

In [ ]:
season_dat['APE'] = abs(season_dat['net_qty'] - season_dat['ecom_marketing_forecast'])/season_dat['net_qty']*100

# season_dat.reset_index(inplace = True)
# season_dat.dropna(inplace = True)


In [ ]:
bins = [0, 100, 10000, 30000]
season_dat['bin'] = pd.cut(np.array(season_dat['net_qty']), bins)

d = season_dat.groupby(['season', 'bin'])['APE'].describe()[['count', 'mean', '50%']]

# --- Sort by season ---

order = {'SS17': 0, 'FW17': 1, 'SS18': 2, 'FW18': 3, 'SS19': 4}
d['order_id'] = [order[i] for i in d.reset_index()['season']]

d.sort_values(by = ['order_id', 'bin'], inplace=True)
d.drop('order_id', axis = 1, inplace=True)

d.round()

### C-MAPE: both corrections

### (1) buy_availability

In [4]:
%%time

dat = pd.read_csv('weekly_dat.csv', low_memory=False, index_col = 0) # Wall time: 10.9 s
dat['consumer_order_date'] = pd.to_datetime(dat['consumer_order_date'])

dat['week'] = [t.week for t in dat['consumer_order_date']]
dat['year'] = [t.year for t in dat['consumer_order_date']]

buy_avail = pd.read_csv('buy_availability.csv', low_memory=False, index_col = 0)

/usr/local/anaconda3/lib/python3.7/site-packages/numpy/lib/arraysetops.py:522: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


CPU times: user 14 s, sys: 617 ms, total: 14.6 s
Wall time: 12.1 s


In [5]:
dat = pd.merge(dat, buy_avail, 
                left_on = ['article_number', 'year', 'week'], 
                right_on = ['article_number', 'year', 'week'])

# net_qty per season
dat_season = pd.DataFrame(
    dat.
    groupby(['article_number', 'season'])['net_qty'].
    sum()
)
dat_season.rename(columns = {'net_qty':'net_qty_season'}, inplace = True)

# WEEKLY averages for articles when fully stocked
dat_stocked = pd.DataFrame(
    dat[dat['avg(buy_availability)'] > 0.35].
    groupby(['article_number', 'season'])['net_qty'].
    mean()
)
dat_stocked.rename(columns = {'net_qty':'net_qty_stocked_weekly_avg'}, inplace= True)

In [6]:
dat_stocked['stocked_season_projection'] = 26*dat_stocked['net_qty_stocked_weekly_avg']

In [7]:
dat_season = (
    pd.merge(dat_season, dat_stocked, left_index=True, right_index=True, how = 'outer').
    round()
)

In [8]:
preds = (pd.DataFrame(
    pd.read_csv('Buyers predictions.csv', low_memory=False, index_col = 0))
         [['season', 'ecom_marketing_forecast']].
         reset_index().
        dropna())

In [9]:
dat_season = pd.merge(
    dat_season, 
    preds, 
    left_index=True, 
    right_on=['article', 'season'], 
    how = 'outer').round().set_index(['article', 'season'])

In [10]:
dat_season.drop('net_qty_stocked_weekly_avg', inplace=True, axis = 1)

dat_season.rename(
    columns = {'stocked_season_projection':'understock_correction', 'net_qty_season': 'net_qty'}, 
    inplace= True)


In [ ]:
del buy_avail, dat, dat_stocked, stocked

### clearance correction

In [24]:
# Wall time: 42.4 s

# dat.to_csv('dat_clearance.csv')
dat_wo_clearance = pd.read_csv('dat_clearance.csv', low_memory=False, index_col = 0) 
dat_wo_clearance['consumer_order_date'] = pd.to_datetime(dat_wo_clearance['consumer_order_date'])

dat_wo_clearance = dat_wo_clearance[dat_wo_clearance['clearance'] != True]
dat_wo_clearance = pd.DataFrame(dat_wo_clearance.groupby(['article_number', 'season'])['net_qty'].sum())

dat_wo_clearance.reset_index(inplace=True)
dat_wo_clearance.rename(columns = {'article_number': 'article', 'net_qty': 'overstock_correction'}, inplace = True)
dat_wo_clearance.set_index(['article', 'season'], inplace = True)

del dat_wo_clearance

In [ ]:
dat_season = pd.merge(dat_season, dat_wo_clearance, 
                             how = 'outer', 
                             left_index = True, 
                             right_index = True)

In [38]:
dat_season.reset_index(inplace = True)

In [39]:
dat_season = dat_season[
    (dat_season['ecom_marketing_forecast'] != 0)
    &
    (dat_season['net_qty'] != 0)
    &
    (dat_season['season'] != 'SS19')
]

In [42]:
def projection(net, forecast, under, over):
    if net > forecast:
        return under
    elif net < forecast:
        return over
    else:
        return under
    


In [46]:
dat_season.head()


,article,season,net_qty,understock_correction,ecom_marketing_forecast,overstock_correction,net_qty*,APE
0,001354,FW17,1474.0,1474.0,1304.0,1434.0,1474.0,11.533243
1,001354,FW18,1805.0,1738.0,2200.0,1805.0,1805.0,21.883657
2,001354,SS17,276.0,598.0,NaN,276.0,598.0,NaN
3,001354,SS18,1271.0,1180.0,1500.0,1254.0,1254.0,19.617225
5,002214,FW17,2662.0,2662.0,2340.0,2676.0,2662.0,12.096168


In [43]:
dat_season['net_qty*'] = dat_season.apply(lambda row: projection(row[2], row[4], row[3], row[5]), axis = 1)
dat_season = dat_season[dat_season['net_qty*'] != 0]

In [45]:
dat_season.loc[:,'APE'] = pd.to_numeric(
    abs(dat_season['net_qty*'] - dat_season['ecom_marketing_forecast'])/dat_season['net_qty*']*100
)


/usr/local/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:362: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
/usr/local/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [49]:
bins = [0, 100, 10000, 30000]
dat_season.loc[:,'bin'] = pd.cut(np.array(dat_season['net_qty']), bins)

d = dat_season.dropna().groupby(['season', 'bin'])['APE'].describe()[['mean', '50%']]

order = {'SS17': 0, 'FW17': 1, 'SS18': 2, 'FW18': 3, 'SS19': 4}
d['order_id'] = [order[i] for i in d.reset_index()['season']]

d.sort_values(by = ['order_id', 'bin'], inplace=True)
d.drop('order_id', axis = 1, inplace=True)

d.round().astype(int)

mean  50%
season bin                      
FW17   (0, 100]         799  327
       (100, 10000]     292   87
       (10000, 30000]    24   22
SS18   (0, 100]        1180  541
       (100, 10000]     184   90
       (10000, 30000]    23   21
FW18   (0, 100]         901  344
       (100, 10000]     236   65
       (10000, 30000]   106   48